# 第八章 分类数据

In [74]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/table.csv')
df.head()

School Class    ID Gender   Address  Height  Weight  Math Physics
0    S_1   C_1  1101      M  street_1     173      63  34.0      A+
1    S_1   C_1  1102      F  street_2     192      73  32.5      B+
2    S_1   C_1  1103      M  street_2     186      82  87.2      B+
3    S_1   C_1  1104      F  street_2     167      81  80.4      B-
4    S_1   C_1  1105      F  street_4     159      64  84.8      B+

## 一. category的创建及其性质

### 1.分类变量的创建

#### a)用Series创建

In [2]:
pd.Series(["a", "b", "c", "a"], dtype="category")

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

#### b)对DataFrame指定类型创建

In [3]:
temp_df = pd.DataFrame({'A':pd.Series(["a", "b", "c", "a"], dtype="category"),'B':list('abcd')})
temp_df.dtypes

A    category
B      object
dtype: object

#### c）利用内置Categorical类型创建

In [4]:
cat = pd.Categorical(["a", "b", "c", "a"], categories=['a','b','c'])
pd.Series(cat)

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

#### d)利用cut函数创建
#### 默认使用区间类型为标签

In [5]:
pd.cut(np.random.randint(0,60,5),[0,10,30,60])

[(0, 10], (30, 60], (10, 30], (10, 30], (0, 10]]
Categories (3, interval[int64]): [(0, 10] < (10, 30] < (30, 60]]

#### 可指定字符为标签

In [6]:
pd.cut(np.random.randint(0,60,5), [0,10,30,60], right=False, labels=['0-10','10-30','30-60'])

[30-60, 10-30, 30-60, 0-10, 30-60]
Categories (3, object): [0-10 < 10-30 < 30-60]

### 2.分类变量的结构

#### 一个分类变量包括三个部分，元素值（values）、分类类别（categories）、是否有序（order）
#### 从上面可以看出，使用cut函数创建的分类变量默认为有序分类变量
#### 下面介绍如何获取或修改这些属性

#### a)describe方法

#### 该方法描述了一个分类序列的情况，包括非缺失值个数、元素值类别数（不是分类类别数）、最多次出现的元素及其频数

In [9]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.describe()

count     4
unique    3
top       a
freq      2
dtype: object

#### b）categories和ordered属性
#### 查看分类类别和是否排序

In [10]:
s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [11]:
s.cat.ordered

False

In [14]:
s.values

[a, b, c, a, NaN]
Categories (4, object): [a, b, c, d]

### 3.类别的修改
#### a)利用set_catgories修改
#### 修改分类，但本身值不会变化

In [28]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s1 = s.cat.set_categories(['new_a','c'])
s

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (4, object): [a, b, c, d]

In [29]:
s1.cat.categories

Index(['new_a', 'c'], dtype='object')

In [30]:
s1.cat.ordered

False

In [31]:
s1.values

[NaN, NaN, c, NaN, NaN]
Categories (2, object): [new_a, c]

#### b）利用rename_categories修改
#### 需要注意的是该方法会把值和分类同时修改

In [33]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.rename_categories(['new_%s'%i for i in s.cat.categories])

0    new_a
1    new_b
2    new_c
3    new_a
4      NaN
dtype: category
Categories (4, object): [new_a, new_b, new_c, new_d]

In [34]:
# 对比set_categories
s.cat.set_categories(['new_%s'%i for i in s.cat.categories])

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
dtype: category
Categories (4, object): [new_a, new_b, new_c, new_d]

#### 利用字典修改值

In [35]:
s.cat.rename_categories({'a':'new_a','b':'new_b'})

0    new_a
1    new_b
2        c
3    new_a
4      NaN
dtype: category
Categories (4, object): [new_a, new_b, c, d]

#### c）利用add_categories添加



In [37]:
s = pd.Series(pd.Categorical(["a", "b", "c", "e",np.nan], categories=['a','b','c','d']))
s.cat.add_categories(['e'])

0      a
1      b
2      c
3    NaN
4    NaN
dtype: category
Categories (5, object): [a, b, c, d, e]

#### d)利用remove_categories移除

In [39]:
s = pd.Series(pd.Categorical(["a", "b", "c", "d",np.nan], categories=['a','b','c','d']))
s.cat.remove_categories(['d'])

0      a
1      b
2      c
3    NaN
4    NaN
dtype: category
Categories (3, object): [a, b, c]

#### e）删除元素值未出现的分类类型

In [40]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.remove_unused_categories()

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (3, object): [a, b, c]

## 二、分类变量的排序

#### 前面提到，分类数据类型被分为有序和无序，这非常好理解，例如分数区间的高低是有序变量，考试科目的类别一般看做无序变量

### 1.序的建立

#### a）一般来说会将一个序列转为有序变量，可以利用as_ordered方法

In [43]:
s = pd.Series(["a", "d", "c", "a"]).astype('category').cat.as_ordered()
s

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a < c < d]

In [44]:
s.cat.ordered

True

In [47]:
s = s.cat.as_unordered()
s

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a, c, d]

In [48]:
s.cat.ordered

False

#### b）利用set_categories方法中的order参数

In [49]:
pd.Series(["a", "d", "c", "a"]).astype('category').cat.set_categories(['a','c','d'],ordered=True)

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a < c < d]

#### c）利用reorder_categories方法

#### 这个方法的特点在于，新设置的分类必须与原分类为同一集合

In [51]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s.cat.reorder_categories(['a','d','c'],ordered=True)

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a < d < c]

In [52]:
#s.cat.reorder_categories(['a','c'],ordered=True) #报错
#s.cat.reorder_categories(['a','c','d','e'],ordered=True) #报错

### 2. 排序

#### 先前在第1章介绍的值排序和索引排序都是适用的

In [82]:
s = pd.Series(np.random.choice(['perfect','good','fair','bad',
                                'awful'],50)).astype('category')
s.cat.set_categories(['perfect','good','fair',
                      'bad','awful'][::-1],ordered=True).head()

0        bad
1    perfect
2        bad
3       good
4       fair
dtype: category
Categories (5, object): [awful < bad < fair < good < perfect]

In [83]:
s.sort_values(ascending=False).head()

24    perfect
12    perfect
34    perfect
35    perfect
37    perfect
dtype: category
Categories (5, object): [awful, bad, fair, good, perfect]

In [84]:
df_sort = pd.DataFrame({'cat':s.values,'value':np.random.randn(50)}).set_index('cat')
df_sort.head()

value
cat              
bad      0.267237
perfect -1.413045
bad     -0.309068
good    -0.630736
fair     1.473768

In [85]:
df_sort.sort_index().head()

value
cat            
awful -0.720408
awful -1.551076
awful  1.800063
awful  2.030382
awful -1.039054

## 三、分类变量的比较操作

### 1. 与标量或等长序列的比较

#### a）标量比较

In [62]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s == 'a'

0     True
1    False
2    False
3     True
dtype: bool

#### b）等长序列比较

In [63]:
s == list('abcd')

0     True
1    False
2     True
3    False
dtype: bool

### 2. 与另一分类变量的比较

#### a）等式判别（包含等号和不等号）
#### 两个分类变量的等式判别需要满足分类完全相同

In [64]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s == s

0    True
1    True
2    True
3    True
dtype: bool

In [65]:
s!=s

0    False
1    False
2    False
3    False
dtype: bool

In [66]:
s_new = s.cat.set_categories(['a','d','e'])

In [68]:
#s == s_new #报错

#### b）不等式判别（包含>=,<=,<,>）
#### 两个分类变量的不等式判别需要满足两个条件：① 分类完全相同 ② 排序完全相同

In [69]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')

In [70]:
s = pd.Series(["a", "d", "c", "a"]).astype('category').cat.reorder_categories(['a','c','d'],ordered=True)
s >= s

0    True
1    True
2    True
3    True
dtype: bool

#### 【问题一】 如何使用union_categoricals方法？它的作用是什么？

In [77]:
from pandas.api.types import union_categoricals
a = pd.Categorical(["a", "b"], ordered=True)
b = pd.Categorical(["a", "b", "a"], ordered=True)
union_categoricals([a, b])
# 链接两个类别序列

[a, b, a, b, a]
Categories (2, object): [a < b]

#### 【问题二】 利用concat方法将两个序列纵向拼接，它的结果一定是分类变量吗？什么情况下不是？

In [80]:
a = pd.Categorical(["a", "b","c"], ordered=True)
b = pd.Categorical(["a", "b", "a"], ordered=True)
#pd.concat([a, b])报错

In [81]:
a = pd.Series(["a", "b", "c", "a"], dtype="category")
b = pd.Series(["a", "b", "c", "a"], dtype="category")
pd.concat([a,b])

0    a
1    b
2    c
3    a
0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

#### 【问题三】 当使用groupby方法或者value_counts方法时，分类变量的统计结果和普通变量有什么区别？

In [92]:
df_sort.groupby("cat")